In [54]:
import pandas as pd 
from bs4 import BeautifulSoup 
import requests 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [55]:
r = requests.get("https://www.nba.com/news/category/kia-race-to-the-mvp-ladder") 
r.status_code

200

In [56]:
url = "https://www.nba.com/news/category/kia-race-to-the-mvp-ladder"
driver = webdriver.Chrome()
driver.get(url)
html = ""

while True:
    try: 
        load_more_button = driver.find_element(By.XPATH, "//button[text()='Load more']")
            
        # Scroll to button and click it
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
        load_more_button.click()
        
        # Wait for content to load and get the HTML
        time.sleep(2) 
        html += driver.page_source
        soup = BeautifulSoup(html, "html.parser")
    except: 
        break

driver.quit()

In [57]:
soup = BeautifulSoup(html, "html.parser") 
title = soup.find_all("a", attrs = {"class": "Anchor_anchor__cSc3P ArticleTile_tileHeadlineLink__9Ed4y"})
time = soup.find_all("div", attrs = {"class": "ArticleTile_tileTimestamp__xz7Ky"})

In [58]:
df = pd.DataFrame({"title": title, "time": time})
df["link"] = df["title"].apply(lambda x: "https://www.nba.com" + x.get("href")) 
df["title"] = df["title"].apply(lambda x: x.get("data-content")) 
df["time"] = df["time"].apply(lambda x: x.get_text()) 
df

,title,time,link
0,Kia MVP Ladder: Tatum rises to No. 3; Jokic st...,43 minutes ago,https://www.nba.com/news/kia-mvp-ladder-march-...
1,Kia MVP Ladder: How Top 5 looks at All-Star break,"February 17, 2023",https://www.nba.com/news/kia-mvp-ladder-feb-17...
2,Kia MVP Ladder: Taking stock after a wild trad...,"February 11, 2023",https://www.nba.com/news/kia-mvp-ladder-feb-10...
3,Kia MVP Ladder: Embiid bests Jokic in key show...,"February 3, 2023",https://www.nba.com/news/kia-mvp-ladder-feb-3-...
4,Kia MVP Ladder: Embiid leaps over Tatum for No. 2,"January 27, 2023",https://www.nba.com/news/kia-mvp-ladder-jan-27...
...,...,...,...
1230,Kia Race to the MVP Ladder: Not even Russell W...,"December 9, 2016",https://www.nba.com/news/mvp-ladder-week-5-man...
1231,Kia Race to the MVP Ladder: Russell Westbrook ...,"December 2, 2016",https://www.nba.com/news/mvp-ladder-week-4-rus...
1232,Kia Race to the MVP Ladder: Anthony Davis craf...,"November 25, 2016",https://www.nba.com/news/mvp-ladder-week-3-ant...
1233,Kia Race to the MVP Ladder: How long until a b...,"November 18, 2016",https://www.nba.com/news/mvp-ladder-week-2-201...


In [68]:
url = "https://www.nba.com/news/kia-mvp-ladder-march-3-2023-edition"
r = requests.get(url) 
soup = BeautifulSoup(r.content, "html.parser") 

title = soup.find("h1").get_text()
date = " ".join(soup.find("time").get_text().split(" ")[:3])
top_5 = list(map(lambda x: x.get_text().split(".")[1].split(",")[0].strip(), soup.find_all("span", attrs = {"style": "font-size: 18pt;"})))
top_6_10 = soup.find_all(lambda tag: tag.name == 'p' and "<strong>6.</strong>" in str(tag))
top_6_10 = BeautifulSoup(str(top_6_10), "html.parser").get_text().split("\n")
top_6_10 = list(map(lambda x: " ".join(x.split(" ")[1:3]).strip(","), top_6_10)) 
ranking = [i for i in range(1, 11)]
top_10 = top_5 + top_6_10

df_ranking = pd.DataFrame({"title": title, "date": date, "ranking": ranking, "player": top_10})
df_ranking

,title,date,ranking,player
0,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",1,Nikola Jokic
1,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",2,Giannis Antetokounmpo
2,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",3,Jayson Tatum
3,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",4,Joel Embiid
4,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",5,Luka Doncic
5,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",6,Kevin Durant
6,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",7,Julius Randle
7,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",8,Domantas Sabonis
8,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",9,Ja Morant
9,Kia MVP Ladder: Jayson Tatum leaps over Joel E...,"March 3, 2023",10,Shai Gilgeous-Alexander
